<a href="https://colab.research.google.com/github/EricSiq/DeepLearning/blob/main/ANN_with_IrisData.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import numpy as np
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder


In [5]:

# Step 1: Define the Neural Network class
class NeuralNetwork:
    def __init__(self, input_size, hidden_size, output_size, learning_rate=0.1):
        """
        Initializes the neural network with specified sizes and learning rate.
        Includes basic error handling for input parameters.
        """
        if not all(isinstance(i, int) and i > 0 for i in [input_size, hidden_size, output_size]):
            raise TypeError("Input, hidden, and output sizes must be positive integers.")
        if not isinstance(learning_rate, (int, float)) or not (0 < learning_rate <= 1):
            raise ValueError("Learning rate must be a number between 0 and 1.")

        self.input_size = input_size
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.learning_rate = learning_rate

        # Step 2: Initialize weights and biases randomly
        # Weights from input to hidden layer
        self.weights_ih = np.random.randn(self.input_size, self.hidden_size) * 0.01
        # Biases for hidden layer
        self.bias_h = np.zeros((1, self.hidden_size))

        # Weights from hidden to output layer
        self.weights_ho = np.random.randn(self.hidden_size, self.output_size) * 0.01
        # Biases for output layer
        self.bias_o = np.zeros((1, self.output_size))


    # Step 3: Define the activation functions (Sigmoid and its derivative)
    def sigmoid(self, x):
        """Sigmoid activation function."""
        return 1 / (1 + np.exp(-x))

    def sigmoid_derivative(self, x):
        """Derivative of the sigmoid function."""
        return x * (1 - x)

    # Step 4: Implement the feedforward process
    def feedforward(self, inputs):
        """
        Passes inputs through the network to get an output.
        Performs basic error handling for input shape.
        """
        if inputs.shape[1] != self.input_size:
            raise ValueError(f"Input shape mismatch. Expected {self.input_size} features, got {inputs.shape[1]}.")

        # Calculate hidden layer output
        self.hidden_layer_input = np.dot(inputs, self.weights_ih) + self.bias_h
        self.hidden_layer_output = self.sigmoid(self.hidden_layer_input)

        # Calculate final output
        self.output_layer_input = np.dot(self.hidden_layer_output, self.weights_ho) + self.bias_o
        self.output = self.sigmoid(self.output_layer_input)

        return self.output

    # Step 5: Implement the backpropagation process
    def backpropagation(self, inputs, targets):
        """
        Calculates and updates weights and biases based on the error.
        Performs basic error handling for input shape.
        """
        if targets.shape != self.output.shape:
            raise ValueError(f"Target shape mismatch. Expected {self.output.shape}, got {targets.shape}.")

        # Calculate output layer error
        output_error = targets - self.output
        output_delta = output_error * self.sigmoid_derivative(self.output)

        # Calculate hidden layer error
        hidden_error = output_delta.dot(self.weights_ho.T)
        hidden_delta = hidden_error * self.sigmoid_derivative(self.hidden_layer_output)

        # Update weights and biases for output layer
        self.weights_ho += self.hidden_layer_output.T.dot(output_delta) * self.learning_rate
        self.bias_o += np.sum(output_delta, axis=0, keepdims=True) * self.learning_rate

        # Update weights and biases for hidden layer
        self.weights_ih += inputs.T.dot(hidden_delta) * self.learning_rate
        self.bias_h += np.sum(hidden_delta, axis=0, keepdims=True) * self.learning_rate

    # Step 6: Define the training loop
    def train(self, inputs, targets, epochs):
        """
        Trains the network for a specified number of epochs.
        Performs basic error handling for input types and shapes.
        """
        if not isinstance(epochs, int) or epochs <= 0:
            raise ValueError("Epochs must be a positive integer.")
        if not isinstance(inputs, np.ndarray) or not isinstance(targets, np.ndarray):
            raise TypeError("Inputs and targets must be numpy arrays.")
        if inputs.shape[0] != targets.shape[0]:
            raise ValueError("Inputs and targets must have the same number of rows.")

        for epoch in range(epochs):
            self.feedforward(inputs)
            self.backpropagation(inputs, targets)

            # Print error every 1000 epochs to monitor progress
            if (epoch + 1) % 1000 == 0:
                loss = np.mean(np.square(targets - self.output))
                print(f"Epoch {epoch + 1}/{epochs}, Loss: {loss:.4f}")

    # Step 7: Define a method for making predictions
    def predict(self, inputs):
        """
        Makes a prediction for given inputs and returns the class label.
        The output is the index of the neuron with the highest activation.
        """
        output = self.feedforward(inputs)
        return np.argmax(output, axis=1)

In [6]:


# Main execution block
if __name__ == '__main__':
    try:
        # Step 8: Prepare and preprocess the Iris dataset
        print("Loading and preprocessing the Iris dataset...")
        iris = load_iris()
        X = iris.data
        y = iris.target.reshape(-1, 1)

        # Normalize the input data to a 0-1 range
        X = X / np.max(X, axis=0)

        # One-hot encode the target labels for multi-class classification
        encoder = OneHotEncoder(sparse_output=False)
        y_one_hot = encoder.fit_transform(y)

        # Split the dataset into training and testing sets
        X_train, X_test, y_train, y_test = train_test_split(X, y_one_hot, test_size=0.2, random_state=42)
        y_test_labels = np.argmax(y_test, axis=1)

        # Step 9: Initialize and train the neural network
        # The network has 4 inputs (features), 8 hidden neurons, and 3 outputs (classes)
        input_size = X_train.shape[1]
        output_size = y_train.shape[1]
        ann = NeuralNetwork(input_size=input_size, hidden_size=8, output_size=output_size, learning_rate=0.5)
        epochs = 10000

        print(f"\nTraining the ANN for Iris classification with {epochs} epochs...")
        ann.train(X_train, y_train, epochs)
        print("Training complete.")

        # Step 10: Evaluate the trained network
        print("\nEvaluating the trained network:")
        predictions = ann.predict(X_test)

        # Calculate accuracy
        accuracy = np.mean(predictions == y_test_labels) * 100
        print(f"Model Accuracy: {accuracy:.2f}%")

    except (TypeError, ValueError) as e:
        print(f"\nAn error occurred during network setup or training: {e}")
    except Exception as e:
        print(f"\nAn unexpected error occurred: {e}")

Loading and preprocessing the Iris dataset...

Training the ANN for Iris classification with 10000 epochs...
Epoch 1000/10000, Loss: 0.1194
Epoch 2000/10000, Loss: 0.1195
Epoch 3000/10000, Loss: 0.0106
Epoch 4000/10000, Loss: 0.0113
Epoch 5000/10000, Loss: 0.0107
Epoch 6000/10000, Loss: 0.0106
Epoch 7000/10000, Loss: 0.0104
Epoch 8000/10000, Loss: 0.0102
Epoch 9000/10000, Loss: 0.0100
Epoch 10000/10000, Loss: 0.0096
Training complete.

Evaluating the trained network:
Model Accuracy: 100.00%
